In [1]:
import os
import sys
import logging
from pathlib import Path

# Add the project root to Python path
project_root = Path().absolute().parent
sys.path.append(str(project_root))

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

print(f"Project root: {project_root}")
print(f"Current working directory: {os.getcwd()}")

Project root: /home/vlad/dev/data-analyser
Current working directory: /home/vlad/dev/data-analyser/notebooks


In [2]:
from src.agent.agent import DataAnalysisAgent
from src.tools.sql_tool import SQLTool
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
agent = DataAnalysisAgent(config_path=str(project_root / "config" / "config.yaml"))

In [ ]:
issue_info = {
    'project': JIRA_PROJECT_KEY,
    'summary': 'Car Models Analysis',
    'description': 'How many unqiue car models we have per car category? Sort the results in descending order!',
    'issuetype': 'Task',
}